In [2009]:
import ast
def print_ast(src):
    print(ast.dump(src, indent=4))
def print_code(src):
    print(ast.unparse(ast.fix_missing_locations(src)))

In [2010]:
def simple_function(x):
    y = 3 * x
    print(y)

In [2011]:
import inspect

In [2012]:
tree = ast.parse(inspect.getsource(simple_function))

In [2013]:
print_ast(tree)

Module(
    body=[
        FunctionDef(
            name='simple_function',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='x')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='y', ctx=Store())],
                    value=BinOp(
                        left=Constant(value=3),
                        op=Mult(),
                        right=Name(id='x', ctx=Load()))),
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='y', ctx=Load())],
                        keywords=[]))],
            decorator_list=[])],
    type_ignores=[])


In [2014]:
print_ast(tree.body[0])

FunctionDef(
    name='simple_function',
    args=arguments(
        posonlyargs=[],
        args=[
            arg(arg='x')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
    body=[
        Assign(
            targets=[
                Name(id='y', ctx=Store())],
            value=BinOp(
                left=Constant(value=3),
                op=Mult(),
                right=Name(id='x', ctx=Load()))),
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Name(id='y', ctx=Load())],
                keywords=[]))],
    decorator_list=[])


In [2015]:
import random

We create a PythonMutator class that links to other Mutator classes, the PythonMutator class tying all of them together with helper functions to call those class' methods. We can later use it to add probabilities etc. to each mutation.

In [2016]:
class PythonMutator(ast.NodeTransformer):
    def __init__(self):
        self.reverse = None
        self.mutations = []        
    
    # def visit_Module(self, src):
    #     return self.generic_visit(src)
    
    # def visit_FunctionDef(self, src):
    #     return self.generic_visit(src)
    
    # def visit_BinOp(self, src):
    #     return self.generic_visit(src)
    
    # def visit_Assign(self, src):
    #     return self.generic_visit(src)
    
    # def visit_Call(self, src):
    #     return self.generic_visit(src)
    
    # def visit_Name(self, src):
    #     return self.generic_visit(src)
    
    # def visit_Constant(self, src):
    #     return self.generic_visit(src)

    def expand_constants(self, src, trials=3):
        node = ExprMutator().modify_value(src, trials)
        self.mutations.extend(node[1])
        return node[0]
    
    def swap_numbers(self, src):
        node = ExprMutator().commute_value(src)
        self.mutations.extend(node[1])
        return node[0]

Now we need to define modify_value that can replace a given constant with an equivalent arithmetic expression, and swap_numbers that will swap the children of a + or * node.

In [2017]:
op_map_int = [("+", ast.Add()), ("*", ast.Mult()), ("//", ast.FloorDiv()), ("-", ast.Sub())]
op_map_float = [("+", ast.Add()), ("*", ast.Mult()), ("/", ast.Div()), ("-", ast.Sub())]

In [2018]:
class ExprMutator(ast.NodeTransformer):
    EXPAND = 1
    COMMUTE = 2

    def __init__(self):
        self.transform = False
        self.trials = 0
        self.mode = self.EXPAND
        self.mutations = []

    def modify_value(self, src, trials):
        self.mode = self.EXPAND
        self.trials = trials
        node = self._modify_value(src)
        return (node, self.mutations)
    
    def _modify_value(self, src):
        if self.trials == 0: return src
        self.transform = True
        self.visit(random.choice(src.body))
        return self._modify_value(src)

We need the mode so we can swap between traversing a path and swapping children. Depth allows us to control how many numbers we want to go and replace with expressions.

In [2019]:
class ExprMutator(ExprMutator):    
    def commute_value(self, n):
        self.mode = self.COMMUTE
        return (self.visit(n), self.mutations)

Now come the real functions. The visits to Constant or BinOp nodes are what will truly handle the functionality.

In [2020]:
from copy import deepcopy

In [2021]:
class ExprMutator(ExprMutator):
    def visit_Constant(self, src):
        if self.transform and self.mode and (isinstance(src.value, int) or isinstance(src.value, float)) == self.EXPAND:
            op_map = {}
            while True:
                try:
                    op = random.randint(0, 3)

                    if isinstance(src.value, int): 
                        op_map = op_map_int
                        other = random.randint(-10000, 10000)
                    else: 
                        op_map = op_map_float
                        other = 500 * random.randint(1, 10) * (random.random() + random.random())
                    
                    assert eval("(" + str(src.value) + op_map[3-op][0] + str(other) + ")" + op_map[op][0] + str(other)) == src.value
                    break
                except ZeroDivisionError: continue
                except AssertionError: continue
            self.trials -= 1
            self.transform = False
            node = ast.fix_missing_locations(ast.BinOp(left = ast.Constant(value=eval("(" + str(src.value) + op_map[3-op][0] + str(other) + ")")), op = op_map[op][1], right = ast.Constant(value=other)))
            #self.mutations update
            self.mutations.append(deepcopy((src, node)))
            return node

        return src

    def visit_BinOp(self, src):
        if self.mode == self.EXPAND:
            if random.randint(1, 2) == 1:
                src.left = self.visit(src.left)
            else:
                src.right = self.visit(src.right)
            return src
        
        if self.mode == self.COMMUTE:
            if isinstance(src.op, ast.Add) or isinstance(src.op, ast.Mult):
                #self.mutations update
                mut = deepcopy(src)
                self.mutations.append((mut, ast.BinOp(left = mut.right, op = mut.op, right = mut.left)))
                src.left, src.right = src.right, src.left
                
            return self.generic_visit(src)

In [2022]:
print_ast(ExprMutator().modify_value(ast.Module(body=[ast.Expr(value=ast.Constant(value=1))]), trials=2)[0])

Module(
    body=[
        Expr(
            value=BinOp(
                left=Constant(value=-191),
                op=Add(),
                right=BinOp(
                    left=Constant(value=7902),
                    op=Sub(),
                    right=Constant(value=7710))))])


In [2023]:
print_code(PythonMutator().expand_constants(ast.parse("x+1")))

x + (-475665731856 // 9877 // 8856 - -5439)


In [2024]:
print_code(PythonMutator().swap_numbers(ast.parse("x + 0.00042426813746287653 * (-5.193317422434368 * 1257 + 8885)")))

(8885 + 1257 * -5.193317422434368) * 0.00042426813746287653 + x


In [2025]:
tree_two = deepcopy(tree)
for i in range(5):
    if random.randint(1, 5) == 1: PythonMutator().swap_numbers(tree_two)
    else: PythonMutator().expand_constants(tree_two)
new_code = ast.unparse(tree_two)
print(new_code)

def simple_function(x):
    y = (39320 - -4759 + -8834 - 155527426340 // 2066 // (-27327973750 // 470 // -6125)) // (14190 - 48570816 // 8144 - 3881 - -9840 + (5069148 // -1813 + -2284)) * x
    print(y)


In [2026]:
simple_function(456)

1368


In [2027]:
exec(new_code)

In [2028]:
simple_function(456)

1368


Clearly the output remains the same inspite of our changes. Next, we look into transforming range-based for loops into while loops.

In [2029]:
for_tree = ast.parse('''for i in range(10, 1, -2):
                        print(i)''')
print_ast(for_tree)

Module(
    body=[
        For(
            target=Name(id='i', ctx=Store()),
            iter=Call(
                func=Name(id='range', ctx=Load()),
                args=[
                    Constant(value=10),
                    Constant(value=1),
                    UnaryOp(
                        op=USub(),
                        operand=Constant(value=2))],
                keywords=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='i', ctx=Load())],
                        keywords=[]))],
            orelse=[])],
    type_ignores=[])


In [2030]:
while_tree = ast.parse('''
i = 10
while i > 1:
    print(i)
    i += -2''')
print_ast(while_tree)

Module(
    body=[
        Assign(
            targets=[
                Name(id='i', ctx=Store())],
            value=Constant(value=10)),
        While(
            test=Compare(
                left=Name(id='i', ctx=Load()),
                ops=[
                    Gt()],
                comparators=[
                    Constant(value=1)]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='i', ctx=Load())],
                        keywords=[])),
                AugAssign(
                    target=Name(id='i', ctx=Store()),
                    op=Add(),
                    value=UnaryOp(
                        op=USub(),
                        operand=Constant(value=2)))],
            orelse=[])],
    type_ignores=[])


In [2031]:
src = for_tree.body[0]
def analyze_for(node):
    args = node.iter.args
    if len(args) == 1:
        return [ast.Constant(value=0), ast.Lt(), args[0], ast.Constant(value=1)]
    elif len(args) == 2:
        return [args[0], ast.Lt(), args[1], ast.Constant(value=1)]
    else:
        step = eval(ast.unparse(args[2]))
        if step < 0:
            return [args[0], ast.Gt(), args[1], args[2]]
        else:
            return [args[0], ast.Lt(), args[1], args[2]]
        
while_args = analyze_for(src)
print_ast(
    ast.Assign(targets=[src.target], value=while_args[0])
    ) 
print_ast(
    ast.While(test=ast.Compare(left=ast.Name(id=src.target.id, ctx=ast.Load()), ops=[while_args[1]], comparators=[while_args[2]]), \
              body=src.body + [ast.AugAssign(target=src.target, op=ast.Add(), value=while_args[3])], orelse=src.orelse)
)

Assign(
    targets=[
        Name(id='i', ctx=Store())],
    value=Constant(value=10))
While(
    test=Compare(
        left=Name(id='i', ctx=Load()),
        ops=[
            Gt()],
        comparators=[
            Constant(value=1)]),
    body=[
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Name(id='i', ctx=Load())],
                keywords=[])),
        AugAssign(
            target=Name(id='i', ctx=Store()),
            op=Add(),
            value=UnaryOp(
                op=USub(),
                operand=Constant(value=2)))],
    orelse=[])


In [2032]:
class PythonMutator(PythonMutator):
    def transform_for(self, src):
        node = ForMutator().transform_for(src)
        self.mutations.extend(node[1])
        return node[0]

In [2033]:
print_ast(ast.parse("for i in delays: print(i)"))
print_ast(ast.parse("for i in range(2): print(i)"))

Module(
    body=[
        For(
            target=Name(id='i', ctx=Store()),
            iter=Name(id='delays', ctx=Load()),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='i', ctx=Load())],
                        keywords=[]))],
            orelse=[])],
    type_ignores=[])
Module(
    body=[
        For(
            target=Name(id='i', ctx=Store()),
            iter=Call(
                func=Name(id='range', ctx=Load()),
                args=[
                    Constant(value=2)],
                keywords=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='i', ctx=Load())],
                        keywords=[]))],
            orelse=[])],
    type_ignores=[])


In [2034]:
class ForMutator(ast.NodeTransformer):
    def __init__(self):
        self.mutations = []

    def transform_for(self, src):
        return (self.visit(src), self.mutations)

    def visit_For(self, src):  
        try: while_args = analyze_for(src)
        except: return src
        node = [ast.Assign(targets=[src.target], value=while_args[0]), \
                ast.While(test=ast.Compare(left=ast.Name(id=src.target.id, ctx=ast.Load()), ops=[while_args[1]], comparators=[while_args[2]]), \
                          body=src.body + [ast.AugAssign(target=src.target, op=ast.Add(), value=while_args[3])], orelse=src.orelse)]
        #self.mutations update
        self.mutations.append((deepcopy(src), deepcopy(node)))
        return node

In [2035]:
for_tree_two = deepcopy(for_tree)
print(ast.unparse(for_tree))
print("====")
print_code(PythonMutator().transform_for(for_tree_two))

for i in range(10, 1, -2):
    print(i)
====
i = 10
while i > 1:
    print(i)
    i += -2


That takes care of for-loops based on ranges. <b>What about iterators?</b>

In [2036]:
print_ast(ast.parse(
'''
L = [1, 4, "hello"]
for i in [len(str(x)) for x in L]:
    print(i)
'''
))

Module(
    body=[
        Assign(
            targets=[
                Name(id='L', ctx=Store())],
            value=List(
                elts=[
                    Constant(value=1),
                    Constant(value=4),
                    Constant(value='hello')],
                ctx=Load())),
        For(
            target=Name(id='i', ctx=Store()),
            iter=ListComp(
                elt=Call(
                    func=Name(id='len', ctx=Load()),
                    args=[
                        Call(
                            func=Name(id='str', ctx=Load()),
                            args=[
                                Name(id='x', ctx=Load())],
                            keywords=[])],
                    keywords=[]),
                generators=[
                    comprehension(
                        target=Name(id='x', ctx=Store()),
                        iter=Name(id='L', ctx=Load()),
                        ifs=[],
                        is_async=0)

In [2037]:
print_ast(ast.parse('x,y = 3 + 5, 3 + 5'))

Module(
    body=[
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='x', ctx=Store()),
                        Name(id='y', ctx=Store())],
                    ctx=Store())],
            value=Tuple(
                elts=[
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5)),
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5))],
                ctx=Load()))],
    type_ignores=[])


In [2038]:
print_ast(ast.parse('''
tmp1, tmp2 = 3 + 5, 3 + 5
x, y = tmp1, tmp2'''))

Module(
    body=[
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='tmp1', ctx=Store()),
                        Name(id='tmp2', ctx=Store())],
                    ctx=Store())],
            value=Tuple(
                elts=[
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5)),
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5))],
                ctx=Load())),
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='x', ctx=Store()),
                        Name(id='y', ctx=Store())],
                    ctx=Store())],
            value=Tuple(
                elts=[
                    Name(id='tmp1', ctx=Load()),
                    Name(id='

The naive method is to copy all targets and rename them on a line above. 

List and Dict subscripts are an issue for this. For example the following AST:

Assign(
    targets=[
        Subscript(
            value=Name(id='gates', ctx=Load()),
            slice=Subscript(
                value=Name(id='inps', ctx=Load()),
                slice=Constant(value=0),
                ctx=Load()),
            ctx=Store())],
    value=Call(
        func=Name(id='float', ctx=Load()),
        args=[
            Subscript(
                value=Name(id='inps', ctx=Load()),
                slice=Constant(value=1),
                ctx=Load())],
        keywords=[]
        )
)

representing 

gates[inps[0]] = float(inps[1])

gets transformed to

xxx[yyy[0]] = float(inps[1])
gates[inps[0]] = xxx[yyy[0]]

but this is problematic because xxx and yyy aren't declared as lists, which they need to be.

To get around this, we need to transform each target to a single variable node.

x, y = 5, 3

must be transformed to

tmp = 5, 3
x, y = tmp

instead.

In [2039]:
class AssignMutator(ast.NodeTransformer):
    def __init__(self, p=100):
        self.p = p
        self.mutations = []

    def transform_assign(self, src):
        return (self.visit(src), self.mutations)
    
    def visit_Assign(self, src):
        if random.randint(1, 100) > self.p: return src
        new_target = ast.Name(id = '_' + str(random.randint(1087345, 196871238674)), ctx = ast.Store())
        node = [ast.Assign(targets = [new_target], value=src.value), ast.Assign(targets=src.targets, value=ast.Name(id=new_target.id, ctx=ast.Load()))]
        #self.mutations update
        self.mutations.append((deepcopy(src), deepcopy(node)))
        return node

In [2040]:
print_ast(AssignMutator().visit(ast.parse('x,y = 3 + 5, 5 + 3')))

Module(
    body=[
        Assign(
            targets=[
                Name(id='_65531242307', ctx=Store())],
            value=Tuple(
                elts=[
                    BinOp(
                        left=Constant(value=3),
                        op=Add(),
                        right=Constant(value=5)),
                    BinOp(
                        left=Constant(value=5),
                        op=Add(),
                        right=Constant(value=3))],
                ctx=Load())),
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='x', ctx=Store()),
                        Name(id='y', ctx=Store())],
                    ctx=Store())],
            value=Name(id='_65531242307', ctx=Load()))],
    type_ignores=[])


In [2041]:
print_ast(ast.parse('x=y=5'))

Module(
    body=[
        Assign(
            targets=[
                Name(id='x', ctx=Store()),
                Name(id='y', ctx=Store())],
            value=Constant(value=5))],
    type_ignores=[])


In [2042]:
print_ast(AssignMutator().visit(ast.parse('x=y=5')))

Module(
    body=[
        Assign(
            targets=[
                Name(id='_181505614132', ctx=Store())],
            value=Constant(value=5)),
        Assign(
            targets=[
                Name(id='x', ctx=Store()),
                Name(id='y', ctx=Store())],
            value=Name(id='_181505614132', ctx=Load()))],
    type_ignores=[])


In [2043]:
print_code(AssignMutator().visit(ast.parse('x,y = 3 + 5, 5 + 3')))

_58372559588 = (3 + 5, 5 + 3)
x, y = _58372559588


In [2044]:
print_code(AssignMutator().visit(ast.parse('x=y=5')))

_186473517892 = 5
x = y = _186473517892


In [2045]:
class PythonMutator(PythonMutator):
    def transform_assign(self, src):
        node = AssignMutator().transform_assign(src)
        self.mutations.extend(node[1])
        return node[0]

In [2046]:
tree = ast.parse(r'''
with open("circuit.txt", "r") as F:
    circuit = F.readlines() # read circuit file into a list
with open("gate_delays.txt", "r") as F:
    delays = F.readlines() # read gate delays into a list

gates = {-1: 0} # prepare dictionary to allow simpler access of gate delays
nodes = {} # prepare dictionary to store node data
out_nodes = [] # prepare list to store names of output nodes
flag1 = flag2 = flag3 = False # prep for processing circuit later

# loop to assign delay value to each kind of gate
for i in delays:
    x = i.strip() # ignore trailing whitespace
    if x[:2] == "//": continue # ignoring whitespace followed by //
    if len(x) == 0: continue # ignoring blank lines or whitespace-only lines
    inps = x.split() # separate line into words
    gates[inps[0]] = float(inps[1]) # assign corresponding delay values with key as gate name

for i in circuit:
    x = i.strip() # ignore trailing whitespace
    if x[:2] == "//": continue # ignoring whitespace followed by //
    if len(x) == 0: continue # ignoring blank lines or whitespace-only lines
    inps = x.split() # separate line into words
    if inps[0] == "PRIMARY_INPUTS": # handling input signal data
        for j in inps[1:]:
            nodes[j] = [0, [], -1] # initializing data with 0 value of delay, no nodes feeding in, associated with no gate  
        flag1 = True # flag to say input signals have been read
        continue
    if inps[0] == "INTERNAL_SIGNALS": # handling internal signal data
        for j in inps[1:]:
            nodes[j] = [0, [], -1] # initializing data with 0 value of delay, no nodes feeding in, associated with no gate
        flag2 = True # flag to say internal signals have been read
        continue
    if inps[0] == "PRIMARY_OUTPUTS": # handling output signal data
        for j in inps[1:]:
            nodes[j] = [0, [], -1] # initializing data with 0 value of delay, no nodes feeding in, associated with no gate
        out_nodes.extend(inps[1:]) # list of output nodes
        flag3 = True # flag to say output signals have been read
        continue
    if flag1 and flag2 and flag3: break # break the loop if all 3 conditions are met before loop termination

for i in circuit: # processing the input and setting up input nodes and gates for each node
    x = i.strip() # ignore trailing whitespace
    if x[:2] == "//": continue # ignoring whitespace followed by //
    if len(x) == 0: continue # ignoring blank lines or whitespace-only lines
    inps = x.split() # separate line into words
    if ((inps[0]=="PRIMARY_INPUTS") or (inps[0]=="INTERNAL_SIGNALS") or (inps[0]=="PRIMARY_OUTPUTS")): 
        continue # ignore signal lines
    out = inps[-1]
    nodes[out][1].extend(inps[1:-1]) # set up input nodes for each node
    nodes[out][2] = inps[0] # set gate delay for relevant nodes

def calcVal_A(x): # recursive function to calculate the delay at each node
    # print(x, nodes) # debug line
    if nodes[x][1] == []: return nodes[x][0] # skip recursive step if node already processed
    s = 0
    for i in nodes[x][1]: # find max delay time of each input node
        nodes[i][0] = calcVal_A(i) # recursive call to function
        s = max(nodes[i][0], s) # node delay that controls delay time of output
    nodes[x][1] = [] # clear input nodes to indicate node delay is already calculated
    return s + gates[nodes[x][2]] # gate delay compensation

to_write = [] # initialize array of lines to be written to output

for i in out_nodes:
    nodes[i][0] = calcVal_A(i) # calculate delay for each output node using the recursive function
    if nodes[i][0] == round(nodes[i][0]): nodes[i][0] = round(nodes[i][0])
    to_write.append(i + " " + str(nodes[i][0]) + "\n") # write delay at each output node to array

with open("output_delays.txt", "w") as F:
    F.writelines(to_write) # write output array to file
''')

In [2047]:
print_code(PythonMutator().expand_constants(PythonMutator().transform_assign(tree)))

with open('circuit.txt', 'r') as F:
    _125383563852 = F.readlines()
    circuit = _125383563852
with open('gate_delays.txt', 'r') as F:
    _187540574859 = F.readlines()
    delays = _187540574859
_152147260537 = {-(572 - 571): 0}
gates = _152147260537
_134254654968 = {}
nodes = _134254654968
_2447854350 = []
out_nodes = _2447854350
_130901645683 = False
flag1 = flag2 = flag3 = _130901645683
for i in delays:
    _106043083333 = i.strip()
    x = _106043083333
    if x[:2] == '//':
        continue
    if len(x) == 0:
        continue
    _142191580891 = x.split()
    inps = _142191580891
    _126756504489 = float(inps[1])
    gates[inps[0]] = _126756504489
for i in circuit:
    _72418067235 = i.strip()
    x = _72418067235
    if x[:8297 - 8295] == '//':
        continue
    if len(x) == 0:
        continue
    _2460976115 = x.split()
    inps = _2460976115
    if inps[0] == 'PRIMARY_INPUTS':
        for j in inps[1:]:
            _77082043675 = [0, [], -1]
            nodes[j] = _77

In [2048]:
import trace
import sys

In [2049]:
def traceit(frame, event, arg):
    """Trace program execution. To be passed to sys.settrace()."""
    if event == 'line':
        global coverage
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        vars = dict(frame.f_locals)
        coverage.append([function_name, lineno, vars])
    return traceit

def tracer(f):
    global coverage
    coverage = []
    sys.settrace(traceit)  # Turn on
    f()
    sys.settrace(None)    # Turn off

In [2050]:
def g():
    def simple_function(x):
        z = 2
        y = 3 * x
        return y
    
    a = simple_function(2)
    b = 0
    for _ in range(6):
        b += 2 * a

    print("The answer is", b)

In [2051]:
tracer(g)

The answer is 72


In [2052]:
for i in coverage:
    print(f"{i[0]} {i[1]} {i[2]}")

g 2 {}
g 7 {'simple_function': <function g.<locals>.simple_function at 0x000002849D8591C0>}
simple_function 3 {'x': 2}
simple_function 4 {'x': 2, 'z': 2}
simple_function 5 {'x': 2, 'z': 2, 'y': 6}
g 8 {'simple_function': <function g.<locals>.simple_function at 0x000002849D8591C0>, 'a': 6}
g 9 {'simple_function': <function g.<locals>.simple_function at 0x000002849D8591C0>, 'a': 6, 'b': 0}
g 10 {'simple_function': <function g.<locals>.simple_function at 0x000002849D8591C0>, 'a': 6, 'b': 0, '_': 0}
g 9 {'simple_function': <function g.<locals>.simple_function at 0x000002849D8591C0>, 'a': 6, 'b': 12, '_': 0}
g 10 {'simple_function': <function g.<locals>.simple_function at 0x000002849D8591C0>, 'a': 6, 'b': 12, '_': 1}
g 9 {'simple_function': <function g.<locals>.simple_function at 0x000002849D8591C0>, 'a': 6, 'b': 24, '_': 1}
g 10 {'simple_function': <function g.<locals>.simple_function at 0x000002849D8591C0>, 'a': 6, 'b': 24, '_': 2}
g 9 {'simple_function': <function g.<locals>.simple_funct

In [2053]:
g_tree = ast.parse(inspect.getsource(g)).body[0]
print_ast(g_tree)

FunctionDef(
    name='g',
    args=arguments(
        posonlyargs=[],
        args=[],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
    body=[
        FunctionDef(
            name='simple_function',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='x')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='z', ctx=Store())],
                    value=Constant(value=2)),
                Assign(
                    targets=[
                        Name(id='y', ctx=Store())],
                    value=BinOp(
                        left=Constant(value=3),
                        op=Mult(),
                        right=Name(id='x', ctx=Load()))),
                Return(
                    value=Name(id='y', ctx=Load()))],
            decorator_list=[]),
   

In [2054]:
print_ast(ast.parse('def f(x, y, *, z=3): print(x)'))

Module(
    body=[
        FunctionDef(
            name='f',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='x'),
                    arg(arg='y')],
                kwonlyargs=[
                    arg(arg='z')],
                kw_defaults=[
                    Constant(value=3)],
                defaults=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Name(id='x', ctx=Load())],
                        keywords=[]))],
            decorator_list=[])],
    type_ignores=[])


In [2055]:
for node in g_tree.body:
    print(node.lineno)

2
7
8
9
12


In [2056]:
def get_trace(src):
    for node in src.body:
        data = src.name, node.lineno
        print(data)
        if isinstance(node, ast.FunctionDef):
            get_trace(node)

In [2057]:
get_trace(g_tree)

('g', 2)
('simple_function', 3)
('simple_function', 4)
('simple_function', 5)
('g', 7)
('g', 8)
('g', 9)
('g', 12)


Now that we can get the line data for each node in the AST, we can get the data of the local variables at a particular AST node and use it for substitutions.

In [2058]:
class VariableInjector(ast.NodeTransformer):  
    def __init__(self):
        self.mutations = []
          
    def traceit(self, frame, event, arg):
        if event == 'line':
            function_name = frame.f_code.co_name
            lineno = frame.f_lineno
            vars = dict(frame.f_locals)
            self.coverage.append([function_name, lineno, vars])
        return self.traceit

    def tracer(self, f):
        self.coverage = []
        sys.settrace(self.traceit)  # Turn on
        f()
        sys.settrace(None)    # Turn off

    def profile_function(self, f, fn_tree = None):
        if fn_tree is None:
            fn_tree = ast.parse(inspect.getsource(f)).body[0]
        self.tracer(f)

        self.seen = set()
        self.unstable = set()
        self.local_vars = set()
        self.browsing = True        
        self.visit(fn_tree)
        
        self.browsing = False
        self.visit(fn_tree)
            
        return (fn_tree, self.mutations)
    
                 

Currenly our class simply combines our existing methods, and then visits the AST. Now what we have to do is, while visiting the AST, we need to find the in-scope variables and their values at every line of execution. Then, we need to look for constants and check if they can be replaced by some variable or some simple arithmetic expression involving a variable.

In [2059]:
class VariableInjector(VariableInjector):
    def visit_Assign(self, src):
        if self.browsing:
            for v in src.targets:
                self.check_seens(v)

        return self.generic_visit(src)
    
    def visit_AugAssign(self, src):
        if self.browsing:
            v = src.target
            self.check_seens(v)

        return self.generic_visit(src)
    
    def visit_For(self, src):
        if self.browsing:
            v = src.target
            self.check_seens(v, True)

        return self.generic_visit(src)
            
    def check_seens(self, v, seen=False):
        if isinstance(v, ast.Tuple):
            for var in v.elts: self.check_seens(var, seen)
        elif isinstance(v, ast.Subscript): self.check_seens(v.value, seen)
        else: 
            if seen: self.seen.add(v.id)
            if v.id in self.seen: self.unstable.add(v.id)
            else: self.seen.add(v.id)
    
    def visit_FunctionDef(self, src):
        self.args = [x.arg for x in src.args.args + src.args.kwonlyargs]
        for node in src.body:
            if not self.browsing: self.get_locals(src.name, node.lineno)
            self.visit(node)
        

Moreover, we want that variables that are assigned lists to be usable by indexing the list. For this, we need to flatten the lists/dicts assigned in our code. This is much of the reason for the visit_Constant function's complexity; if we have a singular value we try substituting that, else we flatten the list and insert tuples of (node, node_value) that will be used for the substitution. 

In [2060]:
print_ast(ast.parse("L[1]"))
print_ast(ast.parse("[1,2,3]"))
print_ast(ast.parse("D['x']"))

Module(
    body=[
        Expr(
            value=Subscript(
                value=Name(id='L', ctx=Load()),
                slice=Constant(value=1),
                ctx=Load()))],
    type_ignores=[])
Module(
    body=[
        Expr(
            value=List(
                elts=[
                    Constant(value=1),
                    Constant(value=2),
                    Constant(value=3)],
                ctx=Load()))],
    type_ignores=[])
Module(
    body=[
        Expr(
            value=Subscript(
                value=Name(id='D', ctx=Load()),
                slice=Constant(value='x'),
                ctx=Load()))],
    type_ignores=[])


In [2061]:
class VariableInjector(VariableInjector):
    def visit_Constant(self, src):
        if len(self.local_vars) == 0 or self.browsing: return src

        queue = list(self.local_vars.keys()).copy()
        random.shuffle(queue)
        while len(queue) != 0:
            n = len(queue) - 1
            if isinstance(queue[n], tuple):
                val = queue[n][1]
                node = queue[n][0]
            else:
                val = self.local_vars[queue[n]]
                node = queue[n]

            queue.pop()
            new_node = None

            if isinstance(val, list) or isinstance(val, tuple):
                rand_val = list(enumerate(val)).copy()
                random.shuffle(rand_val)
                for i in range(len(rand_val)):
                    queue.append((ast.Subscript(value=node, slice=ast.Constant(rand_val[i][0])), rand_val[i][1]))
                
            elif isinstance(val, dict):
                rand_val = list(val.keys()).copy()
                random.shuffle(rand_val)
                for i in rand_val:
                    queue.append((ast.Subscript(value=node, slice=ast.Constant(i)), val[i]))

            else: new_node = self.unify_value(src, node, val)
            
            if new_node is not None: return new_node
            
        return src

Finally, we write the functions to set local_vars and replace basic types with appropriate variable calls.

In [2062]:
class VariableInjector(VariableInjector):
    def get_locals(self, fn, ln):
        self.local_vars = {}
        for i in self.coverage:
            if i[0] == fn and i[1] == ln:
                self.local_vars = {ast.parse(k).body[0].value: v for k, v in i[2].items() if k not in self.args and k not in self.unstable}
                return
        
    def unify_value(self, src, var, val):
        if src.value == val:
            return var
        elif isinstance(src.value, int) and isinstance(val, int) or isinstance(src.value, float) and (isinstance(val, int) or isinstance(val, float)):
            op_map = {}
            try:
                op = random.randint(0, 3)
                if isinstance(src.value, int): op_map = op_map_int
                else: op_map = op_map_float
                assert eval("(" + str(src.value) + op_map[3-op][0] + str(val) + ")" + op_map[op][0] + str(val)) == src.value
                node = ast.BinOp(left = ast.Constant(value=eval("(" + str(src.value) + op_map[3-op][0] + str(val) + ")")), op = op_map[op][1], right = var) 
                #self.mutations update
                self.mutations.append((deepcopy(src), deepcopy(node)))
                return node
            except ZeroDivisionError: return None
            except AssertionError: return None
        elif isinstance(src.value, str) and isinstance(val, str):
            if src.value in val:
                ind = val.find(src.value)
                node = ast.Subscript(value = var, slice = ast.Slice(lower=ast.Constant(value=ind), upper=ast.Constant(value=ind+len(src.value))))
                #self.mutations update
                self.mutations.append((deepcopy(src), deepcopy(node)))
                return node
            elif val in src.value:
                ind = src.value.find(val)
                node = ast.BinOp(left = ast.BinOp(left = ast.Constant(value = src.value[:ind]), op = ast.Add(), right = var), op = ast.Add(), right = ast.Constant(value = src.value[ind + len(val):]))
                #self.mutations update
                self.mutations.append((deepcopy(src), deepcopy(node)))
                return node
        

We have written the functions that traverse the tree and make appropriate calls to functions to get our local variables. Since we are running this entire thing on a function, the outermost scope will always be handled, and then similiarly inner scopes will get handled. One thing we should note is, when using get_locals, we should avoid substituting constants with arguments to the function, because it won't be consistent across function calls.

In [2063]:
print_code(ast.parse(inspect.getsource(g)))
g()

def g():

    def simple_function(x):
        z = 2
        y = 3 * x
        return y
    a = simple_function(2)
    b = 0
    for _ in range(6):
        b += 2 * a
    print('The answer is', b)
The answer is 72


In [2064]:
new_g_code = ast.unparse(VariableInjector().profile_function(g)[0])
print(new_g_code)

The answer is 72
def g():

    def simple_function(x):
        z = 2
        y = (5 - z) * x
        return y
    a = simple_function(2)
    b = -6 + a
    for _ in range(a):
        b += 2 * a
    print('The answer is', b)


In [2065]:
exec(new_g_code)
g()

The answer is 72


Note that using exec to set the value of g currently breaks the VariableInjector because it is unable to find the source code of the function through inspect. Instead, we create a temporary function inside the PythonMutator class and modify that so any code can have variables injected.

In [2066]:
class PythonMutator(PythonMutator):
    sample_tree = ast.parse('''
def pymutator_profile_function():
    pass
''')

    def inject_variables(self, src):
        node = deepcopy(self.sample_tree)
        node.body[0].body = src.body
        node = ast.fix_missing_locations(node)

        current_module = sys.modules[__name__]
        code = compile(node, filename="<ast>", mode="exec")
        exec(code, current_module.__dict__)

        n = VariableInjector().profile_function(pymutator_profile_function, node.body[0])
        self.mutations.extend(n[1])
        
        return ast.Module(body=src.body, type_ignores=src.type_ignores)

In [2067]:
test_str = ast.parse('''
a = "hello world"
print("hello")                     
''')
print_code(PythonMutator().inject_variables(test_str))

hello
a = 'hello world'
print(a[0:5])


In [2068]:
test_list = ast.parse('''
L = [1,2,3]
a=[int(2),int(4),int(6)]
print(a)
print(L)                    
''')

PythonMutator().inject_variables(test_list)
print_code(test_list)

[2, 4, 6]
[1, 2, 3]
L = [1, 2, 3]
a = [int(-1 + L[2]), int(8 // L[1]), int(8 - L[1])]
print(a)
print(L)


In [2069]:
g_code = ast.parse('''
def simple_function(x):
    z = 2
    y = 3 * x
    return y
s = "Hello world!"
a = simple_function(2)
b = 0
for _ in range(6):
    b += 2 * a
print("Hello!")
''')

print_code(g_code)
print("=============")
PythonMutator().expand_constants(g_code)
PythonMutator().expand_constants(g_code)
PythonMutator().inject_variables(g_code)
PythonMutator().expand_constants(g_code)
new_g_code = ast.parse(ast.unparse(g_code))
PythonMutator().inject_variables(new_g_code)
PythonMutator().swap_numbers(new_g_code)
PythonMutator().transform_for(new_g_code)
print_code(g_code)
print("=============")
print_code(new_g_code)

def simple_function(x):
    z = 2
    y = 3 * x
    return y
s = 'Hello world!'
a = simple_function(2)
b = 0
for _ in range(6):
    b += 2 * a
print('Hello!')
Hello!
Hello!
def simple_function(x):
    z = -76 // -38
    y = 3 * x
    return y
s = 'Hello world!'
a = simple_function(-12634 // -6317)
b = (-1710 + a) // (-2 - a) - (219 - a)
for _ in range(-72414 // a + (20502900 // -4275 + a) - -48006 // a - -8864):
    b += (-1355 + 1357) * a
print('Hello!')
def simple_function(x):
    z = -76 // -38
    y = x * (5 - z)
    return y
s = 'Hello world!'
a = simple_function(-12634 // -6317)
b = (a + -(10260 // a)) // (-(12 // a) - a) - (a + 213 - a)
_ = 0
while _ < a + (20502906 - a) // -(a + 4269) + -(a * 12069) // a - -(a * 8001) // a - -(8870 - a):
    b += a * (a + 1351 + -(8130 // a))
    _ += 1
print('Hello!')


Because the equality of python AST nodes is checked by reference address, we need to define a custom function to test equality of two AST nodes. Here it is:

In [2070]:
from itertools import zip_longest
from typing import Union


def compare_ast(node1: Union[ast.expr, list[ast.expr]], node2: Union[ast.expr, list[ast.expr]]) -> bool:
    if type(node1) is not type(node2):
        return False

    if isinstance(node1, ast.AST):
        for k, v in vars(node1).items():
            if k in {"lineno", "end_lineno", "col_offset", "end_col_offset", "parent"}:
                continue
            if not compare_ast(v, getattr(node2, k)):
                return False
        return True

    elif isinstance(node1, list) and isinstance(node2, list):
        return all(compare_ast(n1, n2) for n1, n2 in zip_longest(node1, node2))
    else:
        return node1 == node2

In [2071]:
class PythonMutator(PythonMutator):
    def reverse_mutation(self, src, log=False):
        n = len(self.mutations)
        self.reverse = self.mutations[n-1]
        if log:
            print_code(self.reverse[0])
            print("<==")
            print_code(self.reverse[1])
            print("==")
        self.compare = False
        self.visit(src)
        self.mutations.pop()
        
    def generic_visit(self, src):
        if self.reverse is not None and not self.compare:
            self.compare = True
            test = compare_ast(src, self.reverse[1])
            self.compare = False
            if test:
                src = self.reverse[0]
                self.reverse = None
                return src
        return super().generic_visit(src)

In [2072]:
pm = PythonMutator()
alg_tree = ast.parse("x = 3 * 4")
const_one = alg_tree.body[0].value.left
print_code(pm.expand_constants(alg_tree))
const_two = alg_tree.body[0].value.left

x = 3 * (34460232 // -7988 + 33464500 // 7750)


In [2073]:
print_ast(const_one)
print_ast(const_two)

Constant(value=3)
Constant(value=3)


In [2074]:
pm.mutations.append((const_one, const_two))

In [2075]:
pm.reverse_mutation(alg_tree)
print_code(alg_tree)

x = 3 * (34460232 // -7988 + 33464500 // 7750)


Now that we have found a way to reverse the mutations, all that remains is automatically updating self.mutations, which we have done in the classes.

In [2076]:
pm = PythonMutator()
alg_tree = ast.parse("x = 3 * 4")
pm.expand_constants(alg_tree)
print_code(alg_tree)
for i in range(len(pm.mutations)):
    print_ast(pm.mutations[i][0])
    print("==>")
    print_ast(pm.mutations[i][1])
    print("==")


x = -18996 // -6332 * (-16765511 // -6157 - 2719)
Constant(value=4)
==>
BinOp(
    left=Constant(value=2723),
    op=Sub(),
    right=Constant(value=2719))
==
Constant(value=2723)
==>
BinOp(
    left=Constant(value=-16765511),
    op=FloorDiv(),
    right=Constant(value=-6157))
==
Constant(value=3)
==>
BinOp(
    left=Constant(value=-18996),
    op=FloorDiv(),
    right=Constant(value=-6332))
==


In [2077]:
pm.reverse_mutation(alg_tree)
print_code(alg_tree)
pm.reverse_mutation(alg_tree)
print_code(alg_tree)
pm.reverse_mutation(alg_tree)
print_code(alg_tree)

x = 3 * (-16765511 // -6157 - 2719)
x = 3 * (2723 - 2719)
x = 3 * 4


In [2078]:
pm = PythonMutator()
alg_tree = ast.parse('''
x = 3 * 4
y = 6            
print(y)                
''')
pm.expand_constants(alg_tree)
pm.swap_numbers(alg_tree)
pm.inject_variables(alg_tree)
pm.expand_constants(alg_tree)
pm.swap_numbers(alg_tree)
pm.inject_variables(alg_tree)
print("===")
print_code(alg_tree)
print("===")
for _ in range(3):
    pm.reverse_mutation(alg_tree, True)
    pm.reverse_mutation(alg_tree, True)
    print_code(alg_tree)
    print("===")

6
6
===
x = (-5877 - (-5968 - -88)) * 4
y = (x + (-9425584 + x)) // (-1340 + x - x) + (-3468 // x + (-9129 - x) + -898872 // x) // x
print(y)
===
-74906
<==
-898872 // x
==
-9141
<==
-9129 - x
==
x = (-5877 - (-5968 - -88)) * 4
y = (x + (-9425584 + x)) // (-1340 + x - x) + (-3468 // x + -9141 + -74906) // x
print(y)
===
-289
<==
-3468 // x
==
-1328
<==
-1340 + x
==
x = (-5877 - (-5968 - -88)) * 4
y = (x + (-9425584 + x)) // (-1328 - x) + (-289 + -9141 + -74906) // x
print(y)
===
-9425572
<==
-9425584 + x
==
-9141 + -289
<==
-289 + -9141
==
x = (-5877 - (-5968 - -88)) * 4
y = (x + -9425572) // (-1328 - x) + (-9141 + -289 + -74906) // x
print(y)
===


In [2079]:
while len(pm.mutations) > 0:
    pm.reverse_mutation(alg_tree, True)
    print_code(alg_tree)

-74906 + (-9141 + -289)
<==
-9141 + -289 + -74906
==
x = (-5877 - (-5968 - -88)) * 4
y = (x + -9425572) // (-1328 - x) + (-74906 + (-9141 + -289)) // x
print(y)
-9425572 + x
<==
x + -9425572
==
x = (-5877 - (-5968 - -88)) * 4
y = (-9425572 + x) // (-1328 - x) + (-74906 + (-9141 + -289)) // x
print(y)
(-74906 + (-9141 + -289)) // x + (-9425572 + x) // (-1328 - x)
<==
(-9425572 + x) // (-1328 - x) + (-74906 + (-9141 + -289)) // x
==
x = (-5877 - (-5968 - -88)) * 4
y = (-74906 + (-9141 + -289)) // x + (-9425572 + x) // (-1328 - x)
print(y)
4 * (-5877 - (-5968 - -88))
<==
(-5877 - (-5968 - -88)) * 4
==
x = 4 * (-5877 - (-5968 - -88))
y = (-74906 + (-9141 + -289)) // x + (-9425572 + x) // (-1328 - x)
print(y)
-9430
<==
-9141 + -289
==
x = 4 * (-5877 - (-5968 - -88))
y = (-74906 + -9430) // x + (-9425572 + x) // (-1328 - x)
print(y)
-5880
<==
-5968 - -88
==
x = 4 * (-5877 - -5880)
y = (-74906 + -9430) // x + (-9425572 + x) // (-1328 - x)
print(y)
-84336
<==
-74906 + -9430
==
x = 4 * (-5877 -